In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
train = pd.read_csv('가스.csv', encoding = 'cp949')
test = pd.read_csv('test.csv', sep = '|')

In [4]:
train.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


In [5]:
test.head()

,일자,시간,구분
0,2019-01-01 01 A,NaN,NaN
1,2019-01-01 02 A,NaN,NaN
2,2019-01-01 03 A,NaN,NaN
3,2019-01-01 04 A,NaN,NaN
4,2019-01-01 05 A,NaN,NaN


In [6]:
test['시간'] = test['일자'].str.split(' ').str[1]
test['구분'] = test['일자'].str.split(' ').str[2]
test['일자'] = test['일자'].str.split(' ').str[0]

In [7]:
train['일자'] = pd.to_datetime(train['연월일'])
test['일자'] = pd.to_datetime(test['일자'])

In [8]:
train['월'] = train['일자'].dt.month
test['월'] = test['일자'].dt.month

In [9]:
def get_season(x) :
    if x in [3, 4, 5] :
        return '봄'
    elif x in [6, 7, 8] :
        return '여름'
    elif x in [9, 10, 11] :
        return '가을'
    else :
        return '겨울'

In [10]:
train['계절'] = train.월.apply(get_season)
test['계절'] = test.월.apply(get_season)

In [11]:
train.groupby('계절').공급량.median()

계절
가을     564.985
겨울    1682.721
봄      683.697
여름     360.189
Name: 공급량, dtype: float64

In [12]:
train.groupby('구분').공급량.median().sort_values()

구분
C     120.578
H     178.436
D     492.515
B     676.933
A     904.404
E     968.548
G    1740.981
Name: 공급량, dtype: float64

In [13]:
train['구분'] = train['구분'].map({'G' : 7, 'E' : 6, 'A' : 5, 'B' : 4, 'D' : 3, 'H' : 2, 'C' : 1})
test['구분'] = test['구분'].map({'G' : 7, 'E' : 6, 'A' : 5, 'B' : 4, 'D' : 3, 'H' : 2, 'C' : 1})

In [19]:
train['계절'] = train['계절'].map({'겨울' : 4, '가을' : 2, '여름' : 1, '봄' : 3})
test['계절'] = test['계절'].map({'겨울' : 4, '가을' : 2, '여름' : 1, '봄' : 3})

In [64]:
#train['hot_time'] = [1 if t in [8, 9, 10, 11, 12, 19, 20, 21, 22, 23] else 0 for t in train['시간']]
#test['hot_time'] = [1 if t in [8, 9, 10, 11, 12, 19, 20, 21, 22, 23] else 0 for t in test['시간']]

In [65]:
X = train.drop(['연월일', '일자', '공급량'], axis = 1)
y = train.공급량

In [66]:
target = test[X.columns]

In [67]:
target.시간 = target.시간.astype(int)

In [56]:
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [30]:
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

In [71]:
kf = KFold(n_splits = 15, random_state = 42, shuffle = True)

In [58]:
cat_pred = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(kf.split(X, y))) :
    tr_x, tr_y = X.iloc[idx[0][0]], y.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y.iloc[idx[0][1]]
    
    cat = CatBoostRegressor(iterations = 10000, learning_rate = 0.03, depth = 4, loss_function = 'MAE')
    
    cat.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 2500)

    fold_pred = cat.predict(target) / 15
    cat_pred += fold_pred

0:	learn: 658.3265732	test: 658.3265732	test1: 663.5822290	best: 663.5822290 (0)	total: 81.7ms	remaining: 13m 36s
2500:	learn: 142.3665691	test: 142.3665691	test1: 142.4932376	best: 142.4932376 (2500)	total: 38.1s	remaining: 1m 54s
5000:	learn: 141.3393803	test: 141.3393803	test1: 141.4355568	best: 141.4355568 (5000)	total: 1m 16s	remaining: 1m 16s
7500:	learn: 141.0658618	test: 141.0658618	test1: 141.1786681	best: 141.1786681 (7500)	total: 1m 52s	remaining: 37.5s
9999:	learn: 140.9503868	test: 140.9503868	test1: 141.0915657	best: 141.0910166 (9949)	total: 2m 28s	remaining: 0us

bestTest = 141.0910166
bestIteration = 9949

Shrink model to first 9950 iterations.
0:	learn: 658.4609465	test: 658.4609465	test1: 662.2200330	best: 662.2200330 (0)	total: 18.4ms	remaining: 3m 3s
2500:	learn: 142.2151738	test: 142.2151738	test1: 143.8659095	best: 143.8659095 (2500)	total: 37.3s	remaining: 1m 51s
5000:	learn: 141.1694312	test: 141.1694312	test1: 142.8411917	best: 142.8411917 (5000)	total: 1m 14s

In [72]:
lgbm_pred = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(kf.split(X, y))) :
    tr_x, tr_y = X.iloc[idx[0][0]], y.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y.iloc[idx[0][1]]
    
    lgbm = LGBMRegressor(random_state = 42, max_depth = 5, learning_rate = .05, n_estimators = 10000)
    
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 2500, eval_metric = 'mae')
    
    fold_pred = lgbm.predict(target) / 15
    lgbm_pred += fold_pred

Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[704]	training's l1: 142.999	training's l2: 60074.9	valid_1's l1: 142.082	valid_1's l2: 60133.8
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[876]	training's l1: 142.723	training's l2: 59927.8	valid_1's l1: 145.222	valid_1's l2: 61755.9
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1001]	training's l1: 142.75	training's l2: 60031.9	valid_1's l1: 143.781	valid_1's l2: 59976.7
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1310]	training's l1: 142.613	training's l2: 59650.8	valid_1's l1: 143.852	valid_1's l2: 64965.9
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1003]	training's l1: 142.745	training's l2: 59950.8	valid_1's l1: 143.734	valid_1's l2: 61072.8
Training until validation scores 

In [81]:
submission = pd.read_csv("sample_submission.csv")

In [82]:
submission['공급량'] = (lgbm_pred + cat_pred) / 2

In [87]:
submission.to_csv('3rd.csv', index = False)